In [1]:
import numpy as np

In [3]:
def binary_calculator(digits):
    total = 0
    for idx, exp in enumerate(digits):
        if exp == 1:
            total += 2**(-(idx+1))
        else:
            continue
    return total

In [4]:
def decimal_binary_estimator(number, n_bits, func=binary_calculator):
    ''' Takes a fractional number and returns its binary approximation 
    based on n bit truncation error, along with the actual fractional number estimated by
    the n bit digits. Any number requiring less than n bits will be exact.
    
    Inputs
    -----
    number: float -- The fractional number to be represented
    n_bits: int -- The number of bits to hold the binary representation
    func: function -- Default is the function binary_calculator, which carries out the 
    computation of the the estimate of the number base on an n bit storage of the fractional
    part.
    
    Outputs
    -------
    representation: string -- A string of the binary representation of the input number. 
    result: The estimate of the input based on n bits. Any numbers requiring less than n bits
    to store will be exact.
    '''
    func = binary_calculator
    frac = .5    # seed to allow the while loop to begin
    digits = []
    i = 1        # counter, which will stop at n_bits 
    while (frac != 0.) & (i <= n_bits):
        num = number*2
        integer = int(num)
        #print(f'num: {num}, integer: {integer}, frac: {frac}')
        frac = num % 1
        digits.append(integer)
        number = frac
        i += 1
    result = func(digits)
    binary_rep = [str(x) for x in digits]
    representation = '.'+''.join(binary_rep)
    return representation, result

In [5]:
binary, decimal = decimal_binary_estimator(0.1, 24)

In [6]:
decimal

0.09999996423721313

# The PATRIOT Missile Problem

Per the code above, the PATRIOT missile battery, which used 24 bits to store numbers, has a truncation error associated with attempting to store the repeating binary pattern for the number 0.1.

The absolute relative error for this truncation is:

In [7]:
rel_err = (np.abs(0.1 - decimal))/0.1
rel_err

3.576278687078549e-07

The above result is the error per every 0.1 seconds of operation. Over the course of 100 hours of battery operation, the total error would be:

In [8]:
total_err = rel_err * 100 * 3600
print(f'Total error: {total_err:.2f} seconds')

Total error: 0.13 seconds


The SCUD missile travels 1676 m/s, so the distance the missile could travel after detected by the first pulse of the battery is:

In [9]:
dist = total_err * 1676
print(f'Distance traveled: {dist:.2f} meters')

Distance traveled: 215.78 meters


# Significant digits

We can guarantee at least _m_ significant digits of accuracy if the absolute relative error is:

$$\epsilon_a \leq (0.5x10^{2-m})\%$$

Let's design a code that estimates the sine function up to a desired number of significant digits using the Taylor series expansion for sine:

$$sin(x)\  =\  \sum^{\infty }_{n=0} \frac{(-1)^{n}}{(2n+1)!} x^{2n+1}=x-\frac{x^{3}}{3!} +\frac{x^{5}}{5!} +...$$

In [115]:
def sine_taylor(x, m):
    import math
    '''Funtion to give the approximation to sine out to m significant digits about
    point x using the Taylor Series expansion
    
    Inputs
    ------
    x: float -- The point about which to perform the approximation
    m: int or list -- The number of significant digits desired
    '''
    err = np.inf
    i = 0
    oldval = 0.
    if not isinstance(m, list):
        tol = 0.5*10**(2-m)
        while err >= tol:
            oldval += (-1)**i/math.factorial((2*i+1))*x**(2*i+1)
            n = i + 1
            newval = oldval + (-1)**i/math.factorial((2*i+1))*x**(2*i+1)
            err = np.abs((newval-oldval)/newval)*100
            i += 1
            #oldval = newval
        return i, newval, err
    else:
        i_list = []
        nsig_list = []
        values = []
        for nsigs in m:
            tol = 0.5*10**(2-nsigs)
            while err >= tol:
                oldval += (-1)**i/math.factorial((2*i+1))*x**(2*i+1)
                n = i + 1
                newval = oldval + (-1)**i/math.factorial((2*i+1))*x**(2*i+1)
                err = np.abs((newval-oldval)/newval)*100
                i += 1
            i_list.append(i)
            nsig_list.append(nsigs)
            values.append(newval)
        return list(zip(nsig_list, i_list, values))

In [120]:
m = [1,2,4,6,12]
x = 1.5
results = sine_taylor(x, m)

In [121]:
for sigs, terms, val in results:
    print(f'Significant figures: {sigs}, number of terms: {terms}, value: {val:.{sigs}f}')

Significant figures: 1, number of terms: 4, value: 1.0
Significant figures: 2, number of terms: 4, value: 0.99
Significant figures: 4, number of terms: 6, value: 0.9975
Significant figures: 6, number of terms: 7, value: 0.997495
Significant figures: 12, number of terms: 10, value: 0.997494986604


In [122]:
a = [1,2, 3]
b = [4, 5, 6]
c = [7, 8, 9]
zipped = zip(a,b,c)

# Problem 3 -- Truncation error

The first derivative of a function can be approximated numerically as:

$$f^{\prime }(x)\  \approx \  \frac{f(x+h)-f(x)}{h}$$

which is called the forward Euler approximation. If we let $f(x)=e^x$, find the relative error for $h = (1.0, 0.5, 0.25, 0.125, 0.0625)$.

In [127]:
def exp(x):
    return np.exp(x)

In [129]:
def first_deriv(func, x, h):
    ''' Compute the forward Euler approximation to the first derivative of func'''
    return (func(x+h) - func(x))/h

In [131]:
def relative_error(old, new):
    return np.abs(old-new)/new*100

In [134]:
h = [1., 0.5, 0.25, 0.125, 0.0625]
rel_error_array = np.zeros((len(h),len(h)))

In [158]:
x = 2.
for i, h_old in enumerate(h):
    old = first_deriv(exp, x, h_old)
    for j, h_new in enumerate(h):
        new = first_deriv(exp, x, h_new)
        rel_err = (new-old)/new*100
        rel_error_array[j,i] = rel_err

In [159]:
rel_error_array

array([[  0.        ,  24.49186624,  33.88152933,  38.00856141,
         39.9451635 ],
       [-32.43606354,   0.        ,  12.43530018,  17.900979  ,
         20.46573858],
       [-51.2436676 , -14.20127083,   0.        ,   6.24187467,
          9.17086271],
       [-61.31259779, -21.80413211,  -6.65742265,   0.        ,
          3.12398314],
       [-66.51448214, -25.73197791, -10.09682904,  -3.22472295,
          0.        ]])

In [160]:
import pandas as pd

In [161]:

df = pd.DataFrame(data=rel_error_array,
                 columns=[f'Old h:{h_}' for h_ in h],
                 index=[f'New h:{h_}' for h_ in h])

In [162]:
df

,Old h:1.0,Old h:0.5,Old h:0.25,Old h:0.125,Old h:0.0625
New h:1.0,0.000000,24.491866,33.881529,38.008561,39.945164
New h:0.5,-32.436064,0.000000,12.435300,17.900979,20.465739
New h:0.25,-51.243668,-14.201271,0.000000,6.241875,9.170863
New h:0.125,-61.312598,-21.804132,-6.657423,0.000000,3.123983
New h:0.0625,-66.514482,-25.731978,-10.096829,-3.224723,0.000000


```
# Syntax?
print('hello jupyter')
```